# Download dos dados dos exoplanetas

---

Este notebook mostra como foi feita a seleção dos dados dos exoplanetas disponíveis na base de dados do __[NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html)__, focando em exoplanetas que foram detectados pelos satélites __[Kepler](https://science.nasa.gov/mission/kepler/)__, __[K2](https://science.nasa.gov/mission/kepler/)__ e __[TESS](https://tess.mit.edu/)__.

- João Aires (UFRN)
- Leonardo Andrade de Almeida (ECT/UFRN)

## 0. Bibliotecas

Iniciaremos importando algumas bibliotecas que serão de fundamental importância, são elas:

- ```astropy```: utilizaremos a função ```ascii``` para salvar as tabelas com os dados em arquivos ```.dat```;
- ```astroquery```: utilizaremos a função ```NasaExoplanetArchive``` para acessar remotamente o catálogo de exoplanetas da NASA;
- ```datetime```: utilizaremos para salvar a data em que os arquivos foram criados;
- ```os```: utilizaremos para manipulação de arquivos nos diretórios;
- ```glob```: utilizaremos para listar todos os arquivos com determinado formato no diretório.

In [65]:
# Bibliotecas que são utilizadas
from astropy.io import ascii
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from datetime import datetime
import os
import glob

# Data (formato: YYYYMMDD) em que se está compilando
query_date = datetime.now().strftime('%Y%m%d')
print(f'Data (YYYYMMDD): \t {query_date}')

Data (YYYYMMDD): 	 20241117


## 1. Selecionando colunas relevantes

Para nosso propósito, não necessitaremos de todas as colunas com informações que o _astroquery_ nos fornecerá. Portanto, selecionaremos somente aquelas que serão relevantes para o nosso propósito e as renomearemos para facilitar a utilização futura. As colunas a serem selecionadas (e suas respectivas renomeações) são:

_format: coluna(renomeação)_

- ```pl_name``` _(nome_planeta)_: Nome do exoplaneta;
- ```hostname``` _(nome_estrela)_: Nome da estrela hospedeira;
- ```disc_pubdate``` _(data_publicacao)_: Data (YYYY-MM) da publicação dos parâmetros planetários;
- ```disc_year``` _(ano_deteccao)_: Ano de detecção do exoplaneta;
- ```disc_facility``` _(missao)_: Missão que detectou o exoplaneta (Kepler, K2 ou TESS);
- ```disc_refname``` _(referencia)_: Referência;
- ```ra``` _(ra)_: Ascenção reta;
- ```dec``` _(dec)_: Declinação.

Uma vez que temos as colunas desejadas, criamos uma _string_ contendo seus nomes para facilitar (e reduzir o tempo) no processo de busca remota no catálogo.

In [11]:
# Colunas que serão salvas na tabela
colnames     = ['pl_name', 'hostname', 'disc_pubdate', 'disc_year', 'disc_facility', 'disc_refname', 'ra', 'dec']
colnames_new = ['nome_planeta', 'nome_estrela', 'data_publicacao', 'ano_deteccao', 'missao', 'referencia', 'ra', 'dec']

# Loop para criar a string utilizada no query para selecionar as colunas que serão salvas
col_string = ''
for i,col in enumerate(colnames):
    if i+1 < len(colnames):
        col_string = col_string + f'{col},'
    else:
        col_string = col_string + f'{col}'

print(f'Colunas utilizadas: \t {col_string}')

Colunas utilizadas: 	 pl_name,hostname,disc_pubdate,disc_year,disc_facility,disc_refname,ra,dec


## 2. Busca no catálogo e salvando resultado

Agora que já temos as colunas desejadas para realizar a busca de forma eficiente no catálogo de exoplanetas, vamos realizar esta busca para cada uma das missões espaciais (Kepler, K2 e TESS). Primeiro, excluiremos os arquivos existentes (se houverem).

__Obs.: Caso não queira excluir, pule esta célula.__

In [69]:
list_files = glob.glob('*_exoplanets_*.dat')
for f in list_files:
    os.remove(f)

Uttilizamos a função ```query_criteria``` do ```NasaExoplanetArchive```, passando como input a tabela ```'ps'``` que contém todos os sistemas planetários (como descrito no __[link](https://exoplanetarchive.ipac.caltech.edu/docs/TAP/usingTAP.html)__), as colunas selecionadas e a missão que detectou o exoplaneta (```disc_facility```) juntamente com a flag para selecionar somente aquelas linhas da tabela com o parâmetro default do exoplaneta (```default_flag=1```).

As tabelas contendo os dados finais foram salvas em arquivos ```.dat``` com a seguinte formatação: 

_'missao_exoplanets_YYYYMMDD.dat'_

In [72]:
# Fazendo o download dos exoplanetas descobertos pelo K2 e salvando
k2     = NasaExoplanetArchive.query_criteria(table="ps", select=col_string, where="disc_facility like '%K2%' and default_flag=1")
for co,cn in zip(colnames, colnames_new):
    k2[co].name = cn
filename = 'K2_exoplanets_'+query_date+'.dat'
ascii.write(k2, filename, overwrite=True)
print(f'Exoplanetas do K2 foram salvos em: \t\t {filename}')

# Fazendo o download dos exoplanetas descobertos pelo Kepler e salvando
kepler = NasaExoplanetArchive.query_criteria(table="ps", select=col_string, where="disc_facility like '%Kepler%' and default_flag=1")
for co,cn in zip(colnames, colnames_new):
    kepler[co].name = cn
filename = 'Kepler_exoplanets_'+query_date+'.dat'
ascii.write(kepler, filename, overwrite=True)
print(f'Exoplanetas do Kepler foram salvos em: \t\t {filename}')

# Fazendo o download dos exoplanetas descobertos pelo TESS e salvando
tess   = NasaExoplanetArchive.query_criteria(table="ps", select=col_string, where="disc_facility like '%TESS%' and default_flag=1")
for co,cn in zip(colnames, colnames_new):
    tess[co].name = cn
filename = 'TESS_exoplanets_'+query_date+'.dat'
ascii.write(tess, filename, overwrite=True)
print(f'Exoplanetas do TESS foram salvos em: \t\t {filename}')


# Contagem de objetos nos arquivos 
print('')
print('')

print('Resumo dos dados')
print('(NASA Exoplanet Archive)')
print('-----------------------------------------')
print(f'Kepler: \t\t {len(kepler)} planetas')
print(f'K2: \t\t\t {len(k2)} planetas')
print(f'TESS: \t\t\t {len(tess)} planetas')
print('-----------------------------------------')
print(f'Total: \t\t\t {len(kepler) + len(k2) + len(tess)} planetas')

Exoplanetas do K2 foram salvos em: 		 K2_exoplanets_20241117.dat
Exoplanetas do Kepler foram salvos em: 		 Kepler_exoplanets_20241117.dat
Exoplanetas do TESS foram salvos em: 		 TESS_exoplanets_20241117.dat


Resumo dos dados
(NASA Exoplanet Archive)
-----------------------------------------
Kepler: 		 2778 planetas
K2: 			 548 planetas
TESS: 			 570 planetas
-----------------------------------------
Total: 			 3896 planetas
